### Треугольная платформа на колесах

In [39]:
%load_ext autoreload
%autoreload 2

import sympy as sp
from sympy import Derivative, Symbol, IndexedBase, Idx, Sum, Function, Matrix, Add, Eq
from sympy import diff, symbols, solve, linsolve, simplify, poly, pprint, factor, integrate, lambdify
from sympy import cos, sin, pi

# syspath trick
import sys
sys.path.append('../')

from tatarinov.database.singleton import db
db.set_root('../data/triangular_platform')


from tatarinov.utils.jupyter import display_list, display_obj
# from tatarinov.core.variables import *
from tatarinov.core.tatarinov import TatarinovSystem

from tatarinov.problems.triangular_platform.variables import *
from tatarinov.problems.triangular_platform.structure import *
from tatarinov.problems.triangular_platform.functions import euler, scalar
from tatarinov.utils.sympy import MatrixElDerivative

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Псевдоскорости $\nu_1$, $\nu_2$
#### Координаты $\theta_1$, $\psi_1$, $\alpha$
#### Связи $\psi_1(\nu_1,\nu_2), \theta_1(\nu_1,\nu_2)$

In [2]:
# скорости точек

v['S'] = euler(S, P)
v['P'] = euler(P, C)
v['C'] = euler(C, D)
v['D'] = lambda i: Matrix([0,0,0])

In [3]:
# угловаые скорости

omega['platform']= lambda i: Derivative(alpha,t)*e['z']
omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)

In [36]:
# псевдоскорости nu1, nu2

eq['nu1'] = lambda i: scalar(v['S'](i), e['xi'])
eq['nu2'] = lambda i: scalar(v['S'](i), e['eta'])

eq['nu1(x,y)'] = Eq(nu[1], Derivative(x,t)*cos(alpha) + Derivative(y,t)*sin(alpha))
eq['nu2(x,y)'] = Eq(nu[2], -Derivative(x,t)*sin(alpha) + Derivative(y,t)*cos(alpha))

In [35]:
# nu1(x,y), nu2(x,y)
# x(nu1,nu2), y(nu1,nu2)

eq['diff_x(nu1,nu2)'], eq['diff_y(nu1,nu2)'] = linsolve([eq['nu1(x,y)'], eq['nu2(x,y)']], (Derivative(x, t), Derivative(y, t))).args[0]

In [32]:
# db.save(simplify(Matrix([Eq(nu[1], eq['nu1'](0)),
#                          Eq(nu[2], eq['nu2'](0))])),
#         'nu1,nu2',
#         description='Псевдоскорости nu1, nu2')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


In [14]:
# связи

eq['f(nu1,nu2)'] = lambda i: solve(
                      [Eq(eq['nu1'](i), nu[1]), Eq(eq['nu2'](i), nu[2])],
                      [Derivative(psi[i],t), Derivative(theta[i],t)],
                      dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)

eq['diff(psi)_nu']   = lambda i: eq['f(nu1,nu2)'](i)[Derivative(psi[i],t)]
eq['diff(theta)_nu'] = lambda i: eq['f(nu1,nu2)'](i)[Derivative(theta[i],t)]

In [15]:
# db.save(Matrix([Eq(Derivative(psi[0], t), eq['diff(psi)_nu'](0)),
#                 Eq(Derivative(theta[0], t), eq['diff(theta)_nu'](0))]),
#         'constraints',
#         description='Связи diff(psi), diff(theta) для колеса 0')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


### Лагранжиан для платформы


$$ L = T - V $$ следовательно
$$ T = T_{pl} + T_{fo} + T_{wh} $$ где
$$ T_{pl} = \frac{m_{pl} v_S^2}{2} + \frac{(J_{pl} \omega_{pl}, \omega_{pl})}{2} $$
$$ T_{fo} = 0$$
$$ T_{wh} = \frac{m_{wh} v_S^2}{2} + \frac{(J_{wh} \omega_{wh}, \omega_{wh})}{2} $$

### Платформа

In [16]:
# Момент инерции платформы

J['platform'] = eye(3, 3) * a

In [26]:
# db.save(J['platform'],
#         'J_platform',
#         description='Момент инерации платформы относительно связанной системы отсчета')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


### Колесо

In [17]:
# Момент инерции колесо

J['wheel'] = Matrix([[b,0,0],[0,c,0],[0,0,b]])

In [23]:
# db.save(J['wheel'],
#         'J_wheel',
#         description='Момент инерации колеса относительно связанной системы отсчета')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


_В одинаковых базисах нужно записывать угловую скорость и момент инерции!_

In [19]:
# Угловая скорость колеса в СО связанной с колесом

omega['wheel_in'] = lambda i: psi[i].diff(t)*e['y'] + (theta[i].diff(t) + alpha.diff(t))*e['z']

In [22]:
# db.save(omega['wheel_in'](0),
#         'wheel_in0',
#         description='Угловая скорость колеса в связанной системе отсчета')

### Моменты сил
####  _Платформа_ $$ \vec{M} = -W_0*\vec{e}_z $$ 
####  _Вилки_ $$ \vec{M} = W_0*\vec{e}_z - T_0*\vec{n}_0 $$ 
####  _Колеса_ $$ \vec{M} = T_0*\vec{n}_0 $$ 

In [40]:
# Лагранжиан 

i = 0

pl = m['platform']/2*(Derivative(x, t)**2 + Derivative(y, t)**2)
pl += (J['platform']*omega['platform'](0)).dot(omega['platform'](0)) / 2

wh = m['wheel']/2*(MatrixElDerivative(Matrix([x,y,0]) + SP(i) + PC(i))).dot(MatrixElDerivative(Matrix([x,y,0]) + SP(i) + PC(i)))
wh += (J['wheel']*omega['wheel_in'](0)).dot(omega['wheel_in'](0)) / 2

In [47]:
# db.save(pl,
#         'L_platform',
#         description='Часть Лагранжиана относящееся к платформе')
# db.save(pl,
#         'L_wheel',
#         description='Часть Лагранжиана относящееся к колесу')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform
PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


In [44]:
Lval = Eq(symbols('L'), simplify(pl + wh))
Lval

Eq(L, a*Derivative(alpha(t), t)**2/2 + b*(Derivative(alpha(t), t) + Derivative(theta0(t), t))**2/2 + c*Derivative(psi0(t), t)**2/2 + m1*(Derivative(x(t), t)**2 + Derivative(y(t), t)**2)/2 + m2*((R*sin(beta0 + alpha(t))*Derivative(alpha(t), t) + d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*sin(alpha(t) + theta0(t)) - Derivative(x(t), t))**2 + (R*cos(beta0 + alpha(t))*Derivative(alpha(t), t) + d*(Derivative(alpha(t), t) + Derivative(theta0(t), t))*cos(alpha(t) + theta0(t)) + Derivative(y(t), t))**2)/2)

In [48]:
# db.save(Lval,
#         'L',
#         desciption='Лагранжиан')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


### Связи

In [49]:
constraints = []
constraints += [Eq(Derivative(x, t), eq['diff_x(nu1,nu2)'])]
constraints += [Eq(Derivative(y, t), eq['diff_y(nu1,nu2)'])]
constraints += [Eq(Derivative(alpha, t), nu[3])]
constraints += [Eq(Derivative(theta[0], t), simplify(eq['diff(theta)_nu'](0))).subs({Derivative(alpha, t): nu[3]})]
constraints += [Eq(Derivative(psi[0], t), simplify(eq['diff(psi)_nu'](0))).subs({Derivative(alpha, t): nu[3]})]
display_list(constraints)

Eq(Derivative(x(t), t), nu1(t)*cos(alpha(t)) - nu2(t)*sin(alpha(t)))

Eq(Derivative(y(t), t), nu1(t)*sin(alpha(t)) + nu2(t)*cos(alpha(t)))

Eq(Derivative(alpha(t), t), nu3(t))

Eq(Derivative(theta0(t), t), (-R*nu3(t)*cos(beta0 - theta0(t)) - d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)))/d)

Eq(Derivative(psi0(t), t), (-R*nu3(t)*sin(beta0 - theta0(t)) + nu1(t)*cos(theta0(t)) + nu2(t)*sin(theta0(t)))/r)

In [50]:
# db.save(constraints,
#         'constraints',
#         description='Связи, уже разрешенные относительно старших производных')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform
